<h1>Dependencies</h1>

In [ ]:
!pip install -U demoji sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 7.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylin

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import word_tokenize
import re
from bs4 import BeautifulSoup
import demoji
from sentence_transformers import SentenceTransformer
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Bidirectional, Dense, LSTM
from tensorflow.keras.models import Sequential
from sklearn.metrics import classification_report

<h1>Dataset</h1>

In [ ]:
df = pd.read_csv("kr_train.tsv", sep = "\t")
df_test = pd.read_csv("kr_test.tsv", sep="\t")

In [ ]:
df.head()

,ID,tweet,label
0,kr_train_00001,@user @user @user @user @user @user @user Hhhh...,negative
1,kr_train_00002,"@user Amahano?! Ni impanuka, inkangu, inzara.....",negative
2,kr_train_00003,Ese umuntu aguhaye miliyoni 7 zidorali ngo ary...,negative
3,kr_train_00004,Ugira amagambo😏 kandi Ubwo wasanga nawe byagut...,negative
4,kr_train_00005,Ukuntu inama zose zikomeye zirikubera Mu Rwand...,negative


In [ ]:
df_test.head()

,ID,tweet,label
0,kr_test_00001,@user @user @user Kubeshya ntabwo Ari icyaha u...,neutral
1,kr_test_00002,@user Itegeko ry'umuryango rivuga ko n'umugore...,neutral
2,kr_test_00003,Abafollowers ba byimbisha intugu ba @user @use...,negative
3,kr_test_00004,@user @user Aha niho bita ku kirenge ra?,neutral
4,kr_test_00005,Hari abantu bahisemo brain kuyikoresha nk’amas...,negative


In [ ]:
df.describe()

,ID,tweet,label
count,3302,3302,3302
unique,3302,3259,3
top,kr_train_00001,#ConnectRwanda yageze mu karere ka Rwamagana. ...,neutral
freq,1,2,1257


In [ ]:
df.shape

(3302, 3)

In [ ]:
df.isnull().sum()

ID       0
tweet    0
label    0
dtype: int64

In [ ]:
df["label"].value_counts()

label
neutral     1257
negative    1146
positive     899
Name: count, dtype: int64

In [ ]:
num_words = df["tweet"].apply(lambda x: len(x.split()))
print("Max Words: ", max(num_words))
print("Min Words: ", min(num_words))

Max Words:  69
Min Words:  4


In [ ]:
num_char = df["tweet"].apply(len)
print("Max Chars: ", max(num_char))
print("Min Chars: ", min(num_char))

Max Chars:  458
Min Chars:  23


In [ ]:
def preprocess(data):
    data = data.lower()

    data = re.sub(r'@user','',data) #removing @user mentions in the tweet

    data = re.sub(r'[^a-zA-Z\s]', '', data) #removing symbols/punctuations

    #url management
    data = re.sub(r'http\S+', '', data)
    data = BeautifulSoup(data, 'lxml').get_text()

    data = demoji.replace(data,'') #remove emojis

    # tokens = word_tokenize(data) #tokenize

    return data



In [ ]:
tweet = "@user performed augmentation. more info at https://www.abcd.com"
print(preprocess(tweet))

performed augmentation more info at 


In [ ]:
df["tweet"] = df["tweet"].apply(preprocess)
df.drop(['ID'],inplace=True,axis=1)

In [ ]:
df_test["tweet"] = df_test["tweet"].apply(preprocess)

In [ ]:
df.head()

,tweet,label
0,hhhhhh ntabyihogoza ubu x abo yishe bangana ik,negative
1,amahano ni impanuka inkangu inzara muyite izin...,negative
2,ese umuntu aguhaye miliyoni zidorali ngo arya...,negative
3,ugira amagambo kandi ubwo wasanga nawe byaguta...,negative
4,ukuntu inama zose zikomeye zirikubera mu rwand...,negative


In [ ]:
df.to_csv("preprocessed_train.csv")

In [ ]:
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

In [ ]:
df_test['label'] = label_encoder.fit_transform(df_test['label'])

<h1>Setup and Training</h1>

In [ ]:
model = SentenceTransformer('sentence-transformers/stsb-xlm-r-multilingual')
embeddings = model.encode(df['tweet'])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
test_embeddings = model.encode(df_test["tweet"])

In [ ]:
embeddings

In [ ]:
embeddings.shape

<h3>Linear Embeddings</h3>

In [ ]:
def linear_transformations(embedding):
  random_number = np.random.rand(*embedding.shape) * 0.01
  linear_embeddings = embedding + random_number
  return linear_embeddings

In [ ]:
linear_embeddings = linear_transformations(embeddings)
linear_embeddings_test = linear_transformations(test_embeddings)

<h3>Autoencoder Embeddings</h3>

In [ ]:
class AutoEncoder(tf.keras.Model):
    def __init__(self, embedding_dim, hidden_dim, learning_rate):
        super(AutoEncoder, self).__init__()
        self.encoder = tf.keras.Sequential([
            tf.keras.layers.Dense(hidden_dim, activation='relu', input_shape=(embedding_dim,))
        ])
        self.decoder = tf.keras.Sequential([
            tf.keras.layers.Dense(embedding_dim, activation='linear')
        ])
        self.optimizer = tf.keras.optimizers.Adam(learning_rate)
        self.loss_fn = tf.keras.losses.MeanSquaredError()

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

    def train_step(self, x):
        with tf.GradientTape() as tape:
            reconstructions = self(x)
            loss = self.loss_fn(x, reconstructions)
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        return {'loss': loss}

    def test_step(self, x):
        reconstructions = self(x)
        loss = self.loss_fn(x, reconstructions)
        return {'loss': loss}


In [ ]:
embedding_dim = 768
hidden_dim = 300
learning_rate = 0.001
batch_size = 32

autoencoder = AutoEncoder(embedding_dim, hidden_dim, learning_rate)


dataset = tf.data.Dataset.from_tensor_slices(embeddings).batch(batch_size)

epochs = 10
for epoch in range(epochs):
    for batch in dataset:
        loss = autoencoder.train_step(batch)
    print(f'Epoch {epoch+1}, Loss: {loss["loss"]}')

ae_embeddings = []
for batch in dataset:
    reconstructions = autoencoder(batch)
    ae_embeddings.append(reconstructions.numpy())

ae_embeddings = np.concatenate(ae_embeddings, axis=0)


<h3>Denoising Autoencoder Embeddings</h3>

In [ ]:
class DenoisingAutoEncoder(tf.keras.Model):
    def __init__(self, embedding_dim, hidden_dim, learning_rate):
        super(DenoisingAutoEncoder, self).__init__()
        self.encoder = tf.keras.Sequential([
            tf.keras.layers.Dense(hidden_dim, activation='relu', input_shape=(embedding_dim,))
        ])
        self.decoder = tf.keras.Sequential([
            tf.keras.layers.Dense(embedding_dim, activation='linear')
        ])
        self.optimizer = tf.keras.optimizers.Adam(learning_rate)
        self.loss_fn = tf.keras.losses.MeanSquaredError()

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

    def add_noise(self, x, noise_frac):
        noise = tf.random.normal(tf.shape(x), mean=0.0, stddev=0.1)
        noisy_x = x + noise_frac * noise
        return noisy_x

    def train_step(self, x, noise_frac=0.5):
        with tf.GradientTape() as tape:
            noisy_x = self.add_noise(x, noise_frac)
            reconstructions = self(noisy_x)
            loss = self.loss_fn(x, reconstructions)

        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        return {'loss': loss}

    def test_step(self, x):

        reconstructions = self(x)
        loss = self.loss_fn(x, reconstructions)
        return {'loss': loss}


In [ ]:
embedding_dim = 768
hidden_dim = 300
learning_rate = 0.001
batch_size = 32

denoising_autoencoder = DenoisingAutoEncoder(embedding_dim, hidden_dim, learning_rate)

dataset = tf.data.Dataset.from_tensor_slices(embeddings).batch(batch_size)

epochs = 10
noise_frac = 0.5

for epoch in range(epochs):
    for batch in dataset:
        loss = denoising_autoencoder.train_step(batch, noise_frac)
    print(f'Epoch {epoch+1}, Loss: {loss["loss"]}')


dae_embeddings = []
for batch in dataset:
    reconstructions = denoising_autoencoder(batch)
    dae_embeddings.append(reconstructions.numpy())

dae_embeddings = np.concatenate(dae_embeddings, axis=0)


<h3>Concatenate Embeddings</h3>

In [ ]:
final_embeddings = np.concatenate((linear_embeddings, ae_embeddings, dae_embeddings), axis=1)

<h1>BiLSTM Model for Classification and Metrics Evaluation</h1>

In [ ]:
final_embeddings.shape

(3302, 2304)

In [ ]:
class CustomLSTM(tf.keras.Model):
    def __init__(self, hidden_dim, dropout_rate, learning_rate, num_classes):
        super(CustomLSTM, self).__init__()

        self.hidden_dim = hidden_dim
        self.dropout_rate = dropout_rate
        self.learning_rate = learning_rate
        self.num_classes = num_classes

        self.lstm1 = tf.keras.layers.LSTM(hidden_dim, return_sequences=True)
        self.lstm2 = tf.keras.layers.LSTM(hidden_dim)
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.lstm1(inputs)
        x = self.dropout(x)
        x = self.lstm2(x)
        x = self.dropout(x)
        x = self.fc(x)
        return x

In [ ]:
hidden_dim = 500
dropout_rate = 0.5
learning_rate = 1e-5
num_classes = 3

model = CustomLSTM(hidden_dim, dropout_rate, learning_rate, num_classes)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

model.build(input_shape=(None, 22, 768))

model.summary()


Model: "custom_lstm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               multiple                  2538000   
                                                                 
 lstm_4 (LSTM)               multiple                  2002000   
                                                                 
 dropout_2 (Dropout)         multiple                  0         
                                                                 
 dense_8 (Dense)             multiple                  1503      
                                                                 
Total params: 4541503 (17.32 MB)
Trainable params: 4541503 (17.32 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
